In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import itertools


from sklearn.metrics import confusion_matrix
from tqdm import tqdm


from google.colab import drive
drive.mount('/content/drive')

## Parameters Configuration

In [ ]:
LABELS = ['CompVis_stable-diffusion-v1-4', 'DeepFloyd_IF-II-L-v1.0', 'Real', 'stabilityai_stable-diffusion-2-1-base', 'stabilityai_stable-diffusion-xl-base-1.0']
NUM_OF_CLASSES = len(LABELS)
IMG_SIZE = 224
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

#Adjust the paths as needed
BASE_PATH = '/content/drive/MyDrive/Computer-Vision'
DATASET_PATH = BASE_PATH + '/Dataset/'
MODEL_PATH = BASE_PATH + '/Models/FineTuning/model-8-Epochs.h5'
CSV_FILES_PATHS = [BASE_PATH + '/ChartData/Results-V1-FineTuning-Batch64.csv']

## Accuracy and Loss charts

In [ ]:
#This script will assume to parse CSV files with the following columns:
# - N.EPOCHS
# - ACCURACY (training accuracy)
# - LOSS (training loss)
# - VAL_ACCURACY (validation accuracy)
# - VAL_LOSS (validation loss)

for file_path in CSV_FILES_PATHS:
  model_name = file_path.split("/")[-1].split(".")[0]
  if (model_name.startswith("Results-")):
    model_name = model_name.removeprefix("Results-")

  data = pd.read_csv(file_path, delimiter=";")

  data['ACCURACY'] = data['ACCURACY'] * 100
  data['VAL_ACCURACY'] = data['VAL_ACCURACY'] * 100

  plt.subplots(1,2, figsize=(15,5))
  plt.subplot(121)

  plt.plot(data['N.EPOCHS'], data['ACCURACY'], 'b', linewidth=1.5, label='Training', marker="o", markersize=5)
  plt.plot(data['N.EPOCHS'], data['VAL_ACCURACY'], 'r', linewidth=1.5, label='Validation', marker="o", markersize=5)

  plt.xlabel('Epochs')
  plt.ylabel('Accuracy (%)')
  plt.legend(loc='upper left')
  plt.title("Model Accuracy")

  plt.subplot(122)

  plt.plot(data['N.EPOCHS'], data['LOSS'], 'b', linewidth=1.5, label='Training', marker="o", markersize=5)
  plt.plot(data['N.EPOCHS'], data['VAL_LOSS'], 'r', linewidth=1.5, label='Validation', marker="o", markersize=5)

  loss_min = data[['LOSS', 'VAL_LOSS']].min().min()
  loss_max = data[['LOSS', 'VAL_LOSS']].max().max()
  loss_ticks = np.linspace(loss_min, loss_max, num=10)
  plt.yticks(loss_ticks)

  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend(loc='upper right')
  plt.title("Model loss")

  plt.suptitle(model_name, fontsize=15)

  plt.show()
  plt.clf()

# Confusion Matrix

### Load Model

In [ ]:
model = tf.keras.models.load_model(MODEL_PATH)

### Load Test Data

In [ ]:
def format_image(image , label):
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label

builder = tfds.folder_dataset.ImageFolder(DATASET_PATH)
raw_test = builder.as_dataset(split='test', as_supervised=True)

info = builder.info

print("Total test images: {}  ".format(len(raw_test)) )
print("Label names: {}".format(info.features['label'].names))

test = raw_test.map(format_image)

test_batches = test.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

### Compute Predictions

In [ ]:
true_labels = np.array([], dtype='int16')
predicted_labels = np.array([], dtype='int16')
rejection_threshold = 0.3

for image_batch, label_batch in tqdm(test_batches, total=len(test_batches), desc="Predicting: "):
  for i in range(len(image_batch)):
    image = np.expand_dims(image_batch[i], axis=0)
    true_label = label_batch[i]

    prediction = model.predict(image)
    predicted_class = np.argmax(prediction, axis=1)

    prediction_prob = prediction[0][predicted_class] * 100

    if (prediction_prob < rejection_threshold * 100):
      predicted_class = np.array([5])

    true_labels = np.concatenate((true_labels, [true_label.numpy()]))
    predicted_labels = np.concatenate((predicted_labels, predicted_class))

unknown_count = np.sum(predicted_labels == 5)
total_predictions = len(predicted_labels)
unknown_percentage = (unknown_count / total_predictions) * 100

print(unknown_count)
print(total_predictions)
print(unknown_percentage)

### Matrix Creation

In [ ]:
conf_matrix = confusion_matrix(true_labels.flatten(), predicted_labels.flatten())
conf_matrix_norm = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(7,7))
plt.imshow(conf_matrix_norm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion matrix')
plt.colorbar(shrink=0.7)

tick_marks = np.arange(NUM_OF_CLASSES)
plt.xticks(tick_marks, LABELS, rotation=90)
plt.yticks(tick_marks, LABELS)

thresh = conf_matrix_norm.max() / 2.

for i, j in itertools.product(range(conf_matrix_norm.shape[0]), range(conf_matrix_norm.shape[1])):
    plt.text(j, i, format(conf_matrix_norm[i, j], '.2f'),
            horizontalalignment="center",
            color="white" if conf_matrix_norm[i, j] > thresh else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')

plt.show()